In [ ]:
# Script to search for the in dicom images (in the relevant folders) in the order of their appearance in the csv file

%cd /kaggle/input/siim-covid19-detection/
%matplotlib inline

import pydicom
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pydicom.pixel_data_handlers.util import apply_voi_lut

# Can't get Andres' fun to work
# def read_xray(path, voi_lut = True, fix_monochrome = True):
  #  dicom = pydicom.read_file(path)
    
  #  # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
  #  if voi_lut:
  #      data = apply_voi_lut(dicom.pixel_array, dicom)
  #  else:
  #      data = dicom.pixel_array
               
  #  # depending on this value, X-ray may look inverted - fix that:
  #  if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
  #      data = np.amax(data) - data
        
  #  data = data - np.min(data)
  #  data = data / np.max(data)
  #  data = (data * 255).astype(np.uint8)
        
  #   return data

In [ ]:
# Where are the .dcm files? What do the folders look like?

# We loop over all folders and if it ends with ".dcm" we print it.
pathdf = pd.DataFrame(columns=('root', 'file'))
i = 0
import os
for root, dirs, files in os.walk("."):
    for file in files:
        # print(root)   # ./test/2fb11712bc93/b056067b8455
        # print(file) # a29c5a68b07b.dcm
        # if file.endswith(".dcm"):                              # mixes train and test files
        if file.endswith(".dcm") and root.startswith("./train"): # We only need dcm files and train objects for now
            path_file = os.path.join(root,file)
            #  print(path_file)             # enable to see all the files
            # ./train/cd5dd5e6f3f5/b2ee36aa2df5/d8ba599611e5.dcm [...]
            # These are the paths that will be used to load in the dicom.
            # Don't only print the paths, save them so that we can use them later on. 
            pathdf.loc[i] = [root,file]
            i = i + 1
            
# We have "test" and "train" folders 
# Within we have 2 subfolders
# And then we have the dcm file.
# We probably just need the "train" flders for now

# path_file is the path of one picture, we overwrite the "path_file" each time
# print(len(path_file)) # 50
# type(path_file)       # str
# print(path_file)      # 1 [the last ]path: ./train/22353f3ff2d1/191c168c55c0/058d56025fa2.dcm

In [ ]:
# data.frame of paths
pathdf.head
print(pathdf.shape) # Great: we indeed have 6334 images
pathdf.file


In [ ]:
# Aside from all the pictures, we have 2 csv's : train_image_level and train_study_level

In [ ]:
#1 train_study_level
trainstudy = pd.read_csv('train_study_level.csv')
trainstudy.head(5)
# trainstudy.shape 
# (6054, 5)
# cols: id (unique study identifier), negative, typical, indeterminate, atypical
# rows: 1 row for each study

# these are the study level detections. 
# These are the ouputs in the sample_submission

In [ ]:
# Separate study objects. These are the output ids for the submission file
trainstudy.iloc[:,0]
# _study should be removed later on when we prepare the submission file

In [ ]:
#2 train_image_level: AHA the one we're working with now.
trainimage = pd.read_csv('train_image_level.csv')

trainimage.shape
# (6334, 4)

trainimage.head(5)
# cols: id (6334 unique values), boxes, label, StudyInstanceUiD (6054 unique vals)
# rows: 1 row for each image
# The train data set comprises of 6,334 chest scans. 
# StudyInstance are the ppl? We hebben dus meerdere waarden per study object.
# In the folders we have 6054 directories

# These are the image level predictions. 

# type(trainimage) # dataframe

# BFMA wants to order them in this way



In [ ]:
# There are multiple pictures per study object. There is an Id to link both csvs

trainimage.StudyInstanceUID # last col "StudyInstanceUID" with 6334 inputs
# There's no "_study" at the end of the name

temp = trainimage.StudyInstanceUID.unique() # gives back an array
print(temp) # no underscores here.

len(temp) # 6054
#StudyInstanceImageUnique = np.unique(StudyInstanceImage) # gives back an array
#StudyInstanceImageUnique

# But that's for later, we're here to load in the images. 


In [ ]:
# Each row the trainimage csv corresponds to a figure. How do we call that? 
print(trainimage.id) # col vector of images (eg. id for one row: 000a312787f2_image). 

# Drop the image suffix for the first line [if we are working in a loop]
i = 0                   # indicator to do this for all rows
temp = str(trainimage.id[i])
print(temp)             # 000a312787f2_image
print(temp[0:12])       # 000a312787f2we only take the first 12 letters
print(len(temp[0:12]))  # size 12

# Do this for all images [if we want to make a new variable]
# print(trainimage['id'])
# trainimage_idsubstr = trainimage['id'].str[:12] # only take out first 12 letters to drop "_image" suffix
# print(trainimage_idsubstr)

In [ ]:
# BFMA's new names
newId = ["%04d" % x for x in range(len(trainimage.id))] # 4 digits
# print(newId)
len(newId) # 6334

In [ ]:
# block to look for the correct images in our folders. It prints the path in the order of the csv



# Drop the image suffix for the first line [if we are working in a loop]
i = 0                   # indicator to do this for all rows. We can loop over all rows
# Let's try writing it for one image. It will not work on Kaggle, but it might on BFMA's machine. 

# Image name in the csv
id_csv_i = str(trainimage.id[i])[0:12]
print(id_csv_i)

# Match image name in the csv with the image name in pathdf
# extract the file names wo/ ".dcm" suffix
id_df = pathdf.file.str[:12]
# print(id_df)
# We look for the id in the csv in a list of file names
states = id_csv_i == id_df
j = np.where(states)[0] # position in the df
print(id_df[j])
# Extract the relevant folders
print(pathdf.iloc[j,:])
path = str(pathdf.root.values[j] + '/' + pathdf.file.values[j]) # .values so that we drop the index
print(path) # It adds square brackets. I don't want those square brackets
path = path[2:len(path)-2]
print(path) # N00b solution
type(path) # str

# Let's look at the dicom file
# path = './train/cd5dd5e6f3f5/b2ee36aa2df5/d8ba599611e5.dcm' # manual
# print(path) # ./train/cd5dd5e6f3f5/b2ee36aa2df5/d8ba599611e5.dcm [no square brackets]
# type(path_file)  # str
ds = pydicom.read_file(path)
ds # A lot of meta data
ds.PatientName # anonymized
ds.dir("setup") # empty

# Save under a differnet path
# I can read one image. Let's loop over all images and write them according to a newly chosen name.
suffix = '.dcm'
newName = str(newId[i]) + suffix
print(newName)
# ds.save_as(newName) # Doesn't work on Kaggle.

In [ ]:
# Show images

# Let's use Andres' read_xray. Disabled for now. BFMA requires the real dcims. 

# path = "b74f626d63fa.dcm" # FileNotFound
# path = 'test/2fb11712bc93/b056067b8455/a29c5a68b07b.dcm' # fix path:  Runtime error:  The following handlers are available to decode the pixel data however they are missing required dependencies: GDCM (req. GDCM), pylibjpeg (req. )
#img = read_xray(path, fix_monochrome = False) 
#plt.figure(figsize = (12,12))
#plt.imshow(img, 'gray')